## `public` vs. `private`
```cpp
class Pizza {
    public:
        double hmotnost;
        string druh;
    private:
        string tajna_prisada;
        int kalorie;
};

pizza  = Pizza();

printf("%s\n", pizza.druh); // ok
printf("%s\n", pizza.tajna_prisada); // big no no ~ Cannot access private member
```



In [ ]:
class Pizza:
    def __init__(self):
        self.hmotnost = 200
        self.druh = "margherita"
        self._tajna_prisada = "velrybi tuk"
        self._kalorie = 24000 # kcal/m^2

pizza = Pizza()

pizza._tajna_prisada

- Python private a public nerozlisuje, vse je public
> *We are all consenting adults. (Anyone can touch your privates.)*


# Instance method, class method, static method

In [ ]:
class Pizza():
    testo = {
        "mouka" : [300, "g"],
        "drozdi" : [5, "g"],
        "voda" : [150, "ml"]
    }
    # instance method
    def __init__(self, druh):
        self.hmotnost = 200
        self.druh = druh
        self._tajna_prisada = "velrybi tuk"
        self._kalorie = 24000 # kcal/m^2
        
    def co_je_to_za_pizzu(self):
        print("tuto je", self.druh)
    
    @classmethod # dekorator
    def ukaz_testo(cls):
        print("Prisady:")
        for key, val in cls.testo.items():
            print("{}: {} {}".format(key, val[0], val[1]))
    
    @staticmethod # dekorator
    def co_to_je_pizza():
        print("pizza je kdyz...")

pizza = Pizza("margherita")
pizza.co_je_to_za_pizzu()
Pizza.ukaz_testo()
Pizza.co_to_je_pizza()

### Dekorátory

In [ ]:
def my_dec(func):
    def wrapper():
        print("I am going to say hello:")
        func()
        print("I have just said hello.")
    return wrapper

@my_dec
def say_hello():
    print("Hola")

say_hello()

## Dědičnost (inheritance)

In [ ]:
# zakladni ukazky dedeni
import numpy as np

class Shape():
    def __init__(self, name):
        self.name = name
        self.kind = "some shape"
    
    def area(self):
        pass
    
    def circumference(self):
        pass
    
    def show_stuff(self):
        print(self.name + ", " + self.kind)
        print("Circumference", self.circumference())
        print("Area", self.area())
        print()
        
class Rectangular(Shape):
    def __init__(self, name, a, b):
        super().__init__(name)
        self.kind = "rectangular"
        self.a = a
        self.b = b
        
    def area(self):
        return self.a * self.b
    
    def circumference(self):
        return 2.0 * (self.a + self.b)

class Circle(Shape):
    def __init__(self, name, r):
        super().__init__(name)
        self.kind = "circle"
        self.r = r
        
    def area(self):
        return np.pi * self.r**2
    
    def circumference(self):
        return 2 * np.pi * self.r
    
class Square(Rectangular):
    def __init__(self, name, a):
        super().__init__(name, a, a)
        self.kind = "square"
        
class Polygon(Shape):
    def __init__(self, name, a, n):
        super().__init__(name)
        self.a = a
        self.n = n
        self.kind = "polygon"
        
    def area(self):
        return self.a**2 * self.n / (4 * np.tan(np.pi / self.n))
    
    def circumference(self):
        return self.n * self.a

In [ ]:
tvar = Shape("Karel")
tvar.show_stuff()

obdelnik = Rectangular("Ferdinand", 2, 4)
obdelnik.show_stuff()

square = Square("Leopold", 2)
square.show_stuff()

circle = Circle("Josef", 5)
circle.show_stuff()

test = Polygon("nevim", 4, 4)
test.show_stuff()

## Vlastní výjimky

In [ ]:
from time import sleep
filename = "config.ini"

def create_default_file():
    sleep(1)

try:
    file = open("filename", "r")
except Exception as e:
    print(e)
    print("Creating default file...", end = "")
    create_default_file()
    print("done")
    print("resuming")
    
# zbytek programu

In [ ]:
class ConfigGone(Exception):
    def __init__(self, filename, message = "Config file not found."):
        self.filename = filename
        self.message = message
        super().__init__(message)
    
    def __str__(self):
        return f"{self.filename}: {self.message}"

In [ ]:
from time import sleep
import os.path

filename = "config.ini"

def create_default_file():
    sleep(1)

try:
    if np.random.randint(10) > 8:
        raise Exception("A jeje")
        
    if not os.path.isfile(filename):
        raise ConfigGone(filename)
    
except ConfigGone as e:
    print(e)
    print("Creating default file...", end = "")
    create_default_file()
    print("done")
    print("resuming")
except Exception as e:
    print("nein")


## Observer pattern

In [ ]:
# iterace 1
class Subscriber:
    def __init__(self, name):
        self.name = name
    def update(self, message):
        print('{} got message "{}"'.format(self.name, message))

class Publisher:
    def __init__(self):
        self.subscribers = []
        
    def get_subscribers(self):
        return self.subscribers
    
    def register(self, who):
        self.subscribers.append(who)
        
    def unregister(self, who):
        self.subscribers.remove(who)
        
    def dispatch(self, message):
        for subscriber in self.subscribers:
            subscriber.update(message)
            
sub1 = Subscriber("Karel")
sub2 = Subscriber("Vaclav")

pub = Publisher()
pub.register(sub1)
pub.register(sub2)
pub.dispatch("zprava")
pub.unregister(sub2)
pub.dispatch("zprava2")

In [ ]:
import numpy as np
import time

class FireDetector(Publisher):
    def __init__(self, threshold = 100):
        super().__init__()
        self.threshold = threshold
        self.extinguishing = False
    
    def read_temperature(self):
        return np.random.randint(200)
    
    def run(self):
        for i in range(10):
            temp = self.read_temperature()
            print("Temperature is:", temp)
            if temp > self.threshold and not self.extinguishing:
                self.dispatch(True)
                self.extinguishing = True
            if temp <= self.threshold and self.extinguishing:
                self.dispatch(False)
                self.extinguishing = False
            time.sleep(2)

class Extinguisher(Subscriber):
    def __init__(self, name):
        super().__init__(name)
        
    def update(self, switch):
        if switch:
            print(f"Extinguisher {self.name} is on")
        else:
            print(f"Extinguisher {self.name} is off")

class ThePentagon(Subscriber, Polygon):
    def __init__(self):
        Subscriber.__init__(self, "The Pentagon")
        Polygon.__init__(self, "The Pentagon", 300, 5)
        self.kind = "US Building"
        
    def update(self, switch):
        if switch:
            print("The Pentagon knows there is a fire.")
        else:
            print("The Pentagon knows there is no fire")
        

pentagon = ThePentagon()
pentagon.show_stuff()
det1 = FireDetector()

ext1 = Extinguisher("ext1")
ext2 = Extinguisher("ext2")
det1.register(ext1)
det1.register(ext2)
det1.register(pentagon)

det1.run()

In [ ]:
class Subscriber:
    def __init__(self, name):
        self.name = name
    def update(self, message):
        print('{} got message "{}"'.format(self.name, message))
        
class Publisher:
    def __init__(self, events):
        self.events = { event : dict()
                          for event in events }
        
    def get_subscribers(self, event):
        return self.events[event]
    
    def register(self, event, who, callback=None):
        if callback == None:
            callback = getattr(who, 'update')
        self.get_subscribers(event)[who] = callback
        
    def unregister(self, event, who):
        del self.get_subscribers(event)[who]
        
    def dispatch(self, event, message):
        for subscriber, callback in self.get_subscribers(event).items():
            callback(message)

In [ ]:
sub1 = Subscriber("sub1")
sub2 = Subscriber("sub2")

pub = Publisher(["lunch", "dinner"])
pub.register("lunch", sub1)
pub.register("lunch", sub2)
pub.register("dinner", sub2)
pub.dispatch("lunch", "obed")
pub.dispatch("dinner", "vecere")